In [1]:
import pandas as pd
import numpy as np
import re

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def read_file(relative_path,file_name):
    text= ""
    with open(f'{relative_path}/{file_name}.abc','r') as f:
        text = f.read()
    return text

In [2]:
relative_path ="notebooks/data/final_dataset"
filename_name = 'clean_augmented_data'
#filename_name = 'clean_original_training_data'
#relative_path ="notebooks/data/original_dataset"
training_data_df = load_dataframe(relative_path,filename_name)
training_data_df.columns


Index(['unit_note_length', 'tuneBook', 'title', 'reference_number',
       'original_header', 'original_body', 'meter', 'key', 'clean_song',
       'clean_header', 'clean_body', 'chord_progression', '"Gm"', '"G7"',
       '"G"', '"F7"', '"F#m"', '"F#7"', '"F"', '"Em"', '"Eb"', '"E7"', '"E"',
       '"Dm"', '"D7"', '"D"', '"Cm"', '"C7"', '"C"', '"Bm"', '"Bb"', '"B7"',
       '"B"', '"Am"', '"A7"', '"A"'],
      dtype='object')

In [3]:
training_data_df.tail()

,unit_note_length,tuneBook,title,reference_number,original_header,original_body,meter,key,clean_song,clean_header,...,"""Cm""","""C7""","""C""","""Bm""","""Bb""","""B7""","""B""","""Am""","""A7""","""A"""
9492,1/4,dataset_min5.abc,Grandpa's,78,X:78\nT:Grandpa's\nM:4/4\nL:1/4\nK:A,"A/2G/2|""D""FA""A7""Bc|""D""d/2c/2d/2e/2fa|""Em""gf""E7...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""|""Em""""E7""|""A7""|...","M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""|""Em""""E7""|""A7""|...",...,0,0,0,0,0,0,0,0,7,2
9493,1/4,dataset_min5.abc,The Girl With The Green Hat On,79,X:79\nT:The Girl With The Green Hat On\nM:4/4\...,"(3A/2B/2c/2|""D""dA""A7""A/2B/2A/2G/2|""D""F/2G/2A/2...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""""A7""|""D""|""A7""|""...","M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""""A7""|""D""|""A7""|""...",...,0,0,0,0,0,0,0,0,6,2
9494,1/4,dataset_min5.abc,Green Meadow,80,X:80\nT:Green Meadow\nM:4/4\nL:1/4\nK:D,"(3D/2E/2F/2|""G""GG/2A/2B/2G/2B/2d/2|""C""e/2f/2g/...",4/4,D,"M:4/4\nL:1/4\nK:D\n|""G""|""C""""G""|""Am""|""Am""""D7""|""...","M:4/4\nL:1/4\nK:D\n|""G""|""C""""G""|""Am""|""Am""""D7""|""...",...,0,0,6,0,0,0,0,4,0,0
9495,1/4,dataset_min5.abc,The Old Grey Cat,82,X:82\nT:The Old Grey Cat\nM:4/4\nL:1/4\nK:Bm,"B|""Em""eeEE/2F/2|""Em""G/2F/2G/2A/2B/2A/2B/2^c/2|...",4/4,Bm,"M:4/4\nL:1/4\nK:Bm\n|""Em""|""Em""|""D""|""D""|""Em""|""E...","M:4/4\nL:1/4\nK:Bm\n|""Em""|""Em""|""D""|""D""|""Em""|""E...",...,0,0,0,0,0,3,0,1,0,0
9496,1/4,dataset_min5.abc,Gyre And Gimble,84,X:84\nT:Gyre And Gimble\nM:4/4\nL:1/4\nK:A,"A|""D""dAFA|""Em""BG""A7""EG|""D""FAd3/2e/2|""A7""f/2g/2...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""|""Em""""A7""|""D""|""A7""""D""|""...","M:4/4\nL:1/4\nK:A\n|""D""|""Em""""A7""|""D""|""A7""""D""|""...",...,0,0,0,0,0,0,0,0,7,1


In [4]:
training_data_df["clean_header"].str.len().max()

332

In [5]:
bodies = ""
silences = 0
for body in training_data_df["clean_body"]:
    if 'z' in body:
        silences +=1 
    bodies += body+"\n"
chars = sorted(list(set(bodies)))
vocab_size = len(chars)
print('vocab: ',''.join(chars))
print('vocab_size',vocab_size)
print("silences ",silences)

vocab:  
"#'(),-/1234567=ABCDEFG[]^_abcdefgmz|~
vocab_size 39
silences  516


In [6]:
training_data_text = read_file(relative_path,filename_name)

print("number of chars:",len(training_data_text))

number of chars: 4020982


In [7]:
chars = sorted(list(set(training_data_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
45


In [8]:
import torch

print(torch.__version__)

2.0.0+cu117


In [9]:
import transformers
print(transformers.__version__)

4.28.0.dev0


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import wandb
import tiktoken

print(wandb.__version__)

0.14.2


In [3]:
!ls

Dockerfile  docker-compose.yaml  overrides.json
README.md   notebooks		 requirements.txt


In [4]:
from dotenv import load_dotenv
import os
nano_path = 'notebooks/nanoGPT'
os.chdir(nano_path)
load_dotenv()

True

In [5]:
!ls

LICENSE      config	      out-abc-char	    transformer_sizing.ipynb
README.md    configurator.py  out-shakespeare-char  wandb
__pycache__  data	      sample.py
assets	     model.py	      scaling_laws.ipynb
bench.py     older_ckpt       train.py


In [ ]:
# Dataset with multiple voices present
#length of dataset in characters: 4,149,703
#all the unique characters: 
#"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
#vocab size: 46
#train has 3,734,732 tokens
#val has 414,971 tokens

In [6]:
!python3 data/abc_char/prepare.py

length of dataset in characters: 4,020,980
all the unique characters: 
"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
vocab size: 45
train has 3,618,882 tokens
val has 402,098 tokens


In [7]:
!python3 train.py config/train_abc_char.py

Overriding config with config/train_abc_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-abc-char'
eval_interval = 25 # keep frequent because we'll overfit
eval_iters = 500
log_interval = 20 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'abc-char'
wandb_run_name = 'mini-char-gpt'

dataset = 'abc_char'
batch_size = 64
block_size = 512 # context of up to 512 previous characters

# baby GPT model :)
n_layer = 8
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 5 # not super necess

In [8]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 14.18M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
d2|"G"B3/2g/2d3/2B/2G3/2B/2d3/2B/2|"D"c3/2e/2a3/2g/2f3/2d/2e3/2f/2|"G"B3/2g/2d3/2B/2g3/2d/2B3/2d/2|"D"c3/2A/2d3/2A/2e3/2A/2f3/2A/2|"G"g3/2d/2B3/2d/2g3/2b/2a3/2g/2|"D"f3/2d/2A3/2d/2f3/2a/2g3/2f/2|"C"e3/2d/2c3/2B/2"D"c3/2e/2d3/2c/2|"G"B2G2G2d2|"D"ADBDcDdc|"G"BGcG^cGdG|"D"ADBDcDd2|"G"edd^cd4|"D"ADBDcDdc|"G"BGcG^cGd2|"C"ecgc"D"fcac|"G"g2b2g4|]

M:4/4
L:1/4
K:A
|"D"|"G""A"|"D"|"Em""A"|"D"|"G""A"|"G""A"|"D"|"G""D"|"G""D"|"G""D"|"E""A"|"G""D"|"G""D"|"G""D"|"E""A"|"A""D"|]
A/2|"D"d/2c/2d/2e/2fA|"G"Be"A
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
d2|"G"g2d2|"C"ee"G"d4|"C"cBA2G|"D"A2-|"G"Bdd2|"C"efg"G"d4|"C"ee"G"ef|"C"gfe"G"dcB|"D"A2"G"G2|"G"Bdd2|"C"efg"G"d2|"C"cBAc"G"BAGB|"D"A2F2|"G"GBdg2|"C"egBc|"G"dBg"C"edcB|"Am"ABcA"D"DFA2|"G"GBdgdBd|"C"edcece

In [31]:
 !python3 sample.py --out_dir=older_ckpt/m_voices --path_meta=older_ckpt/m_voices --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = older_ckpt/m_voices
Overriding: path_meta = older_ckpt/m_voices
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 14.18M
shakespeare_char
Loading meta from older_ckpt/m_voices/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb/2e/2|"D"f/2e/2f/2g/2a/2f/2a/2f/2|"G"g/2f/2g/2a/2"C"g/2e/2d/2e/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb|"D"d'd'"D7"e/2f3/2|"G"g3/2a/2gG/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][GB]G/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2c/2|"G"B/2G/2

In [29]:
cat older_ckpt/m_voices/ckpt.pkl

cat: older_ckpt/m_voices/ckpt.pkl: No such file or directory


In [30]:
!ls -l older_ckpt/m_voices

total 168544
-rwxrwxrwx 1 1000 1000 172577338 Apr  9 16:07 ckpt.pt
-rwxrwxrwx 1 1000 1000       513 Apr  9 17:01 meta.pkl


In [6]:
!ls -l out-abc-char/

total 168536
-rw-r--r-- 1 root root 172577338 Apr  9 03:10 ckpt.pt


In [9]:
!id

uid=0(root) gid=0(root) groups=0(root)
